<a href="https://colab.research.google.com/github/carlosprr29/ai-progetto-spagnoli/blob/main/notebooks/03_1_BERT_Training_WELFake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets torch scikit-learn
!pip install --upgrade transformers

In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Load dataset
dataset = load_dataset("davanstrien/WELFake")
df = dataset["train"].to_pandas()

# Clean
df = df.dropna(subset=["title", "text"])

# Stratified 80/20 division
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

In [ ]:
def train_bert(train_text, test_text, train_y, test_y, name):

    print(f"\n Training BERT for: {name}")

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    train_encodings = tokenizer(
        train_text.tolist(),
        truncation=True,
        padding=True,
        max_length=128
    )

    test_encodings = tokenizer(
        test_text.tolist(),
        truncation=True,
        padding=True,
        max_length=128
    )

    class DatasetTorch(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
            item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    train_dataset = DatasetTorch(train_encodings, train_y.tolist())
    test_dataset = DatasetTorch(test_encodings, test_y.tolist())

    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=2, # 3 is more robust
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        fp16=True,
        do_eval=True,
        logging_dir="./logs",
        save_strategy="no",
        load_best_model_at_end=False
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=1)
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=1)

    acc = accuracy_score(test_y, preds)

    print(f"\n✅ Accuracy ({name}): {acc:.4f}")
    print(classification_report(test_y, preds))

    #We return the rating, the trained AI, and the translator that converts
    #your words into the specific numbers that model understands.
    return acc, model, tokenizer

In [ ]:
#We carry out the training
acc_titulo, model_tit, token_tit = train_bert(
    train_df["title"],
    test_df["title"],
    train_df["label"],
    test_df["label"],
    "TITLES ONLY"
)

# We save the trained AI and the translator
model_tit.save_pretrained('./model_WELFake_titles')
token_tit.save_pretrained('./model_WELFake_titles')

# Connect and copy to Drive
from google.colab import drive
drive.mount('/content/drive')
import shutil
# We copy it into the folder.
shutil.copytree('./model_WELFake_titles', '/content/drive/MyDrive/Project_IA/model_WELFake_titles')

In [ ]:
# We take a small sample so as not to overload the RAM.
from sklearn.model_selection import train_test_split

train_df_sample, _ = train_test_split(
    train_df,
    train_size=6000,
    stratify=train_df["label"],
    random_state=42
)

test_df_sample, _ = train_test_split(
    test_df,
    train_size=1500,
    stratify=test_df["label"],
    random_state=42
)

train_df = train_df_sample
test_df = test_df_sample


# We carry out the training
train_df["total"] = train_df["title"] + " " + train_df["text"]
test_df["total"] = test_df["title"] + " " + test_df["text"]

acc_total, model_full, token_full = train_bert(
    train_df["total"],
    test_df["total"],
    train_df["label"],
    test_df["label"],
    "TITLE + TEXT"
)

# We save the trained AI and the translator.
model_full.save_pretrained('./model_WELFake_full')
token_full.save_pretrained('./model_WELFake_full')

# Connect and copy to Drive
from google.colab import drive
drive.mount('/content/drive')
import shutil
# We copy it to Drive.
shutil.copytree('./model_WELFake_full', '/content/drive/MyDrive/Project_IA/model_WELFake_full')

In [ ]:
print(" RESULTS OF THE ABLATION STUDY")
print("-" * 30)
# Replace acc_title with the value given to you during the first training session
print(f"Accuracy (Titles Only):   {0.9584:.4f}")
print(f"Accuracy (Title + Text): {0.9833:.4f}")

difference = (0.9833 - 0.9584) * 100
print("-" * 30)
print(f"✅ Conclusion: Adding the full text improves the model by {difference:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# --- DATA ---
acc_real_titles = 0.9584
acc_real_text = 0.9833

labels = ['Titles Only', 'Title + Text']
values = [acc_real_titles, acc_real_text]

# --- CREATE GRAPH ---
fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(labels, values, color=['#A0C4FF', '#023E8A'])

# Visual settings
ax.set_ylim(0.85, 1.0)
ax.set_ylabel('Accuracy')
ax.set_title('Impact of Full Text on Detection (Ablation Study)')

# Add the percentages above the bars.
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.005, f"{yval*100:.2f}%",
            ha='center', va='bottom', fontweight='bold')

plt.show()